This notebook plots the output of the script:
`python run_kpsrecoman.py --input-dlmerged [merged_dlreco.root] --input-larflow [larmatchme_larlite.root] -o test_prepsp.root -n 1 -e 0 -tb -mc --stop-after-spacepointprep`

Running with the `--stop-after-spacepointprep` halts the reco chain after the spacepoints have been split by ssnet track/shower, split by in-time/out-of-time wirecell tagger, and have been filtered to maximize larmatch score per plane pixel.

This notebook takes in:
 * the larmatch output larlite file that stores the output of the larmatch network
 * the keypoint output larlite file that stores the output of run_keypointreco.py
 * optionally, if the mcinfo or dlmerged files is provided that contains truth meta data, then one can also plot the true trajectories of particles

In [ ]:
from __future__ import print_function
import os,sys
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import torch
import ROOT as rt
from larlite import larlite
from ROOT import larutil
import lardly

In [ ]:
# SETUP THE GEOMETRY

# Set the detector in larlite
detid = larlite.geo.kICARUS
#detid = larlite.geo.kMicroBooNE
#detid = larlite.geo.kSBND
larutil.LArUtilConfig.SetDetector(detid)

# Get geometry class
geo = larlite.larutil.Geometry.GetME()

# Get detector outlines from lardly
from lardly.detectors.getdetectoroutlines import DetectorOutlineFromLarlite
detoutline = DetectorOutlineFromLarlite(detid)
detlines = detoutline.getlines()

In [ ]:
# define some colors for particle types
kp_color_array = np.array( ((255,0,0),    # nu
                            (0,255,0),    # track-start
                            (0,0,255),    # track-end
                            (255,0,255),  # shower
                            (0,255,255),  # shower-michel
                            (255,255,0)),     # shower-delta
                            dtype=np.float64 ) 

In [ ]:
PLOT_MC = False

In [ ]:
# file lists

# The input file
kpreco_output_larlite_v = ["../test_prepsp_larlite.root"]


In [ ]:
io = larlite.storage_manager( larlite.storage_manager.kREAD )
for f in kpreco_output_larlite_v:
    io.add_in_filename( f )
io.open()

In [ ]:
# Get entry and relevant data
ENTRY = 0
io.go_to(ENTRY)

ev_larmatch_hits = io.get_data( larlite.data.kLArFlow3DHit, "larmatch" )
norig_larmatch_hits = ev_larmatch_hits.size()
print("[Entry] ",ENTRY,": number of larmatch hits=",norig_larmatch_hits)

if PLOT_MC:
    # Get MC info
    mcinfoplots = lardly.data.visualize_nu_interaction(io, do_sce_correction=True )


In [ ]:
# Plot larmatch hits
lfplot = lardly.data.visualize_larlite_larflowhits( ev_larmatch_hits, "larmatch", score_threshold=0.5)
plot_traces = detlines+[lfplot]

if PLOT_MC:
    # Get MC info
    mcinfoplots = lardly.data.visualize_nu_interaction(io, do_sce_correction=True )
    plot_traces += mcinfoplots


# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='All input larmatch hits',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_traces, layout=layout)
fig.show()

In [ ]:
# Plot post-filter shower hits
# These are hits that have high ssnet shower score and pass the WireCell in-time filter
# They also have ghost-removal through max-plane larmatch score filter

ev_finalshower_hits = io.get_data( larlite.data.kLArFlow3DHit, "maxshowerhit" )
finalshower_hit_plot = lardly.data.visualize_larlite_larflowhits( ev_finalshower_hits, "maxshowerhit", score_threshold=0.5)
finalshower_hit_traces = detlines+[finalshower_hit_plot]
print("number of hits in 'maxshowerhit': ",ev_finalshower_hits.size())

if PLOT_MC:
    finalshower_hit_traces += mcinfoplots


# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Final in-time shower spacepoints',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=finalshower_hit_traces, layout=layout)
fig.show()

In [ ]:
# Plot post-filter track hits
# These are hits that have high ssnet TRACK score and pass the WireCell in-time filter
# They also have ghost-removal through max-plane larmatch score filter

ev_finaltrack_hits = io.get_data( larlite.data.kLArFlow3DHit, "maxtrackhit_wcfilter" )
finaltrack_hit_plot = lardly.data.visualize_larlite_larflowhits( ev_finaltrack_hits, "maxtrackhit_wcfilter", score_threshold=0.5)
finaltrack_hit_traces = detlines+[finaltrack_hit_plot]
print("number of hits in 'maxtrackhit_wcfilter': ",ev_finaltrack_hits.size())

if PLOT_MC:
    finaltrack_hit_traces += mcinfoplots


# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Final in-time track spacepoints',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=finaltrack_hit_traces, layout=layout)
fig.show()

In [ ]:
# Plot post-filter out-of-time track
# These are hits that have high ssnet TRACK score and pass the WireCell in-time filter
# They also have ghost-removal through max-plane larmatch score filter

ev_outoftime_track_hits = io.get_data( larlite.data.kLArFlow3DHit, "full_maxtrackhit" )
outoftime_track_hit_plot = lardly.data.visualize_larlite_larflowhits( ev_outoftime_track_hits, "full_maxtrackhit", score_threshold=0.5)
outoftime_track_hit_traces = detlines+[outoftime_track_hit_plot]
print("number of hits in 'full_maxtrackhit' (cosmic track hits): ",ev_outoftime_track_hits.size())

if PLOT_MC:
    outoftime_track_hit_traces += mcinfoplots


# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Final out-of-time track spacepoints',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=outoftime_track_hit_traces, layout=layout)
fig.show()

In [ ]:
# NOTES